In [1]:
import sys

sys.path.append('../GSL')

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import pickle

from torch_geometric.data import Data as gData
from torch_geometric.utils import to_networkx, to_undirected
from torch_geometric.nn import MessagePassing

import networkx as nx
import matplotlib.pyplot as plt

from torch_geometric_temporal.nn.recurrent import DCRNN

In [3]:
from torch_geometric.utils import to_undirected, to_dense_adj, add_self_loops, sort_edge_index, remove_self_loops

# GTS

In [4]:
from models.GTS.gts_graph_learning2 import GTS_Graph_Learning
from models.GTS.gts_forecasting_module import GTS_Forecasting_Module
from models.GTS.self_attention_graph_learning import Attention_Graph_Learning
from models.GTS.DCRNN import DCRNN
from utils.utils import build_fully_connected_edge_idx, build_batch_edge_index
from dataset.make_traffic_dataset import TrafficDatasetLoader
from torch_geometric.utils import dense_to_sparse
from models.MTGNN.mtgnn_graph_learning import MTGNN_Graph_Learning

from glob import glob
import yaml
from easydict import EasyDict as edict

In [5]:
config_file = glob('../GSL/config/GTS/*.yaml')[0]
config = edict(yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader))

In [6]:
loader = TrafficDatasetLoader(raw_data_dir=config.dataset.root, dataset_name=config.dataset.name)
dataset,entire_inputs = loader.get_dataset(num_timesteps_in=config.encoder_step,
                                                 num_timesteps_out=config.decoder_step,
                                                 batch_size=config.train.batch_size)

In [7]:
entire_inputs.shape

torch.Size([325, 2, 52105])

In [ ]:
config.graph_learning.sampling = 'attention'

In [8]:
edge_index = build_fully_connected_edge_idx(num_nodes=config.nodes_num)

In [9]:
edge_index.shape

torch.Size([2, 105625])

In [11]:
gl = GTS_Graph_Learning(config)
al = Attention_Graph_Learning(config)
ml = MTGNN_Graph_Learning(config)

In [ ]:
gts_style = gl(entire_inputs, edge_index)
attention = al(entire_inputs)
low_rank_approx = ml()